In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir("/content/drive/MyDrive/ADM/HW5/")

In [3]:
import networkx as nx
from tqdm import tqdm

##EXPLORE THE DATASET AND FIND MAXIMUM AND MINIMUM TIMESTAMPS

In [ ]:
a2q=open("a2q.txt")
max=0
min=504796

for line in tqdm(a2q):
    
    #Read the line
    splitted_line=line.split(" ")
    interaction_time=int(int(splitted_line[2])/3600)
    if interaction_time > max:
      max=interaction_time
    if interaction_time < min:
      min=interaction_time

print(max)
print(min)


17823525it [00:22, 781062.94it/s]

404796
338213


In [ ]:
c2q=open("c2q.txt")
max=0
min=504796

for line in tqdm(c2q):
   
    #Read the line
    splitted_line=line.split(" ")
    interaction_time=int(int(splitted_line[2])/3600)
    if interaction_time > max:
      max=interaction_time
    if interaction_time < min:
      min=interaction_time

print(max)
print(min)

20268151it [00:25, 781426.87it/s]

404798
338231


In [ ]:
c2a=open("c2a.txt")
max=0
min=504796

for line in tqdm(c2a):
    
    #Read the line
    splitted_line=line.split(" ")
    interaction_time=int(int(splitted_line[2])/3600)
    if interaction_time > max:
      max=interaction_time
    if interaction_time < min:
      min=interaction_time

print(max)
print(min)

25405374it [00:32, 774312.59it/s]

404798
338234


#Read data

In [4]:
# def namestr(obj, namespace):
#     return([name for name in namespace if namespace[name] is obj][0])

In [4]:
def create_graph_layer(G,file_type,file_name,include_self_loops,maximum_time):
  
  for line in tqdm(file_type):
    
    #Read the line
    splitted_line=line.split(" ")
    #Extract source node,destination node and interaction time
    source_node=int(splitted_line[0])
    destination_node=int(splitted_line[1])
    interaction_time=int(int(splitted_line[2])/3600)

    #Disregard self loops if required and take only first maximum_time hours of the dataset
    if ((source_node!=destination_node) or include_self_loops) and interaction_time<maximum_time:
      
      #If G has already an edge, increase its weight and update the interaction time
      if G.has_edge(source_node,destination_node) and file_name in G[source_node][destination_node]:
        G[source_node][destination_node][file_name][file_name]+=1
        G[source_node][destination_node][file_name]["time"]=interaction_time
      
      #If that edge is noy yet existing, initialize its type and value depending on the file we are reading
      else:
        if file_name=="a2q":
          G.add_edge(source_node, destination_node,key=file_name, a2q=1,time=interaction_time)
        if file_name=="c2q":
          G.add_edge(source_node, destination_node,key=file_name, c2q=1,time=interaction_time)
        if file_name=="c2a":
          G.add_edge(source_node, destination_node,key=file_name, c2a=1,time=interaction_time)
  return(G)

In [14]:
#Open dataset files
a2q=open("a2q.txt")
c2a=open("c2a.txt")
c2q=open("c2q.txt")

#Define timestamp offset in hours
offset=338213

#Create multidirected graph
G = nx.MultiDiGraph()
G=create_graph_layer(G,c2a,"c2a",include_self_loops=False,maximum_time=offset+1000)
G=create_graph_layer(G,a2q,"a2q",include_self_loops=False,maximum_time=offset+1000)
G=create_graph_layer(G,c2q,"c2q",include_self_loops=False,maximum_time=offset+1000)

#Close dataset files
a2q.close()
c2a.close()
c2q.close()

25405374it [00:40, 622709.62it/s]
17823525it [00:29, 609149.05it/s]
20268151it [00:32, 622529.13it/s]
